<a href="https://colab.research.google.com/github/0jipy/192kbps_colab_JIPY/blob/main/__04_TextGeneration_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM을 이용한 텍스트 생성 (GRU)
 - 데이터 다운로드: 캐글 New York Times Comments
 - ArticlesApril2018.csv

In [1]:
import numpy as np 
import tensorflow as tf
seed=2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

- 뉴욕타임즈 기사 

In [3]:
import pandas as pd
# df = pd.read_csv('data/ArticlesApril2018.csv', encoding='latin1')
df = pd.read_csv('/content/ArticlesApril2018.csv', encoding='latin1')
df.head(3)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...


- 데이터 전처리

In [ ]:
df.columns

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [4]:
# headline 열 데이터만 사용, Null 검사
df.headline.isnull().sum()

0

In [5]:
headlines = [text for text in df.headline.values]
len(headlines)

1324

In [6]:
headlines[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [7]:
# 노이즈 데이터('Unknown') 제거
headlines = [text for text in headlines if text != 'Unknown']
len(headlines)

1214

In [8]:
# 구둣점 제거, 소문자로 변환
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
def preprocessing(s):
    s = s.encode('utf8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [10]:
headlines = [preprocessing(s) for s in headlines]
headlines[:3]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained']

In [11]:
# 단어 집합을 만들고, 그 크기를 확인
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) + 1
vocab_size

3494

In [12]:
# 학습에 사용될 샘플 시퀀스
sequences = []
for line in headlines:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [13]:
max_len = max(len(s) for s in sequences)
max_len

24

In [14]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          99,  269],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   99,
         269,  371],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   99,  269,
         371, 1115]], dtype=int32)

In [15]:
X = sequences[:,:-1]
y = sequences[:,-1]
Y = to_categorical(y)
X.shape, Y.shape

((7803, 23), (7803, 3494))

## 모델 정의/설정/학습
- Embedding
- LSTM
- GRU 쓰지 않나?

In [23]:
# del model

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [25]:
# 임베딩 벡터 10차원, LSTM 노드 크기 128 = >GRU로변경
model = Sequential([ 
    Embedding(vocab_size, 10, input_length=max_len-1),  ## ??
    GRU(128),
    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 23, 10)            34940     
                                                                 
 gru (GRU)                   (None, 128)               53760     
                                                                 
 dense_1 (Dense)             (None, 3494)              450726    
                                                                 
Total params: 539,426
Trainable params: 539,426
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

In [27]:
%time history = model.fit(X, Y, epochs=200, verbose=1)

Epoch 1/200
244/244 [==============================] - 3s 6ms/step - loss: 7.6597 - accuracy: 0.0302
Epoch 2/200
244/244 [==============================] - 2s 6ms/step - loss: 7.1099 - accuracy: 0.0324
Epoch 3/200
244/244 [==============================] - 1s 6ms/step - loss: 6.8862 - accuracy: 0.0466
Epoch 4/200
244/244 [==============================] - 1s 6ms/step - loss: 6.6500 - accuracy: 0.0499
Epoch 5/200
244/244 [==============================] - 1s 6ms/step - loss: 6.3897 - accuracy: 0.0550
Epoch 6/200
244/244 [==============================] - 1s 6ms/step - loss: 6.1206 - accuracy: 0.0619
Epoch 7/200
244/244 [==============================] - 1s 6ms/step - loss: 5.8435 - accuracy: 0.0651
Epoch 8/200
244/244 [==============================] - 1s 6ms/step - loss: 5.5663 - accuracy: 0.0720
Epoch 9/200
244/244 [==============================] - 1s 6ms/step - loss: 5.3035 - accuracy: 0.0804
Epoch 10/200
244/244 [==============================] - 1s 6ms/step - loss: 5.0482 - accura

## 모델 검증

In [28]:
from my_util import sentence_generation

In [29]:
print(sentence_generation(model, t, max_len, 'i', 10))
print(sentence_generation(model, t, max_len, 'epa', 10))
print(sentence_generation(model, t, max_len, 'former', 10))

i want to be rich and im not sorry it just
epa chiefs ethics woes have echoes in his past of privatized
former nfl cheerleaders settlement offer 1 and a meeting with goodell
